In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Define the directory and filenames
image_dir = 'bariloche2'
filenames = ['B1.tif', 'B2.tif', 'B3.tif', 'B4.tif', 'B5.tif', 'B6.tif', 'B7.tif', 'B8.tif']

# Read and stack images to create a multi-band image
bands = []
for filename in filenames:
    filepath = os.path.join(image_dir, filename)
    img = cv2.imread(filepath, cv2.IMREAD_UNCHANGED)
    bands.append(img)
multiband_image = np.dstack(bands)

# Convert the first band to 8-bit for display purposes
display_image = cv2.convertScaleAbs(bands[0], alpha=(255.0/np.max(bands[0])))

# Initialize lists to hold samples and labels
samples = []
labels = []

print("Select regions of different textures for training. Press ESC to finish.")

while True:
    # Allow user to select ROI
    roi = cv2.selectROI("Select ROI and press ENTER or SPACE. Press ESC to exit.", display_image)
    x, y, w, h = roi
    if w == 0 or h == 0:
        break  # Exit loop if no selection is made

    # Extract pixels from the selected region
    roi_pixels = multiband_image[y:y+h, x:x+w, :].reshape(-1, multiband_image.shape[2])
    samples.append(roi_pixels)

    # Get label input from user
    label = input("Enter label for selected region: ")
    labels.append(np.full((roi_pixels.shape[0],), label))
    

cv2.destroyAllWindows()

print("Concatenating...")
# Concatenate all samples and labels
X = np.vstack(samples)
y = np.hstack(labels)

print("Encoding...")
# Encode string labels to integers
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print("Splitting...")
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

print("Training...")
# Train SVM classifier
clf = svm.SVC()
clf.fit(X_train, y_train)

print("Predicting...")
# Predict on test set and compute confusion matrix
y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Apply the trained classifier to the entire image
height, width, n_bands = multiband_image.shape
image_reshaped = multiband_image.reshape(-1, n_bands)
predicted_labels = clf.predict(image_reshaped)

# Convert predicted labels back to original labels
classified_image_labels = le.inverse_transform(predicted_labels)
classified_image = classified_image_labels.reshape(height, width)

# Map labels to integers for visualization
label_to_int = {label: idx for idx, label in enumerate(le.classes_)}
int_to_label = {idx: label for label, idx in label_to_int.items()}
classified_image_int = np.vectorize(label_to_int.get)(classified_image)

# Display the classified image
plt.figure(figsize=(10, 10))
plt.imshow(classified_image_int, cmap='jet')
plt.title('Classified Image')
plt.colorbar(ticks=range(len(le.classes_)), label='Classes')
plt.clim(-0.5, len(le.classes_)-0.5)
plt.show()


Select regions of different textures for training. Press ESC to finish.
Concatenating...
Encoding...
Splitting...
Trainning...
Predicting...
Confusion Matrix:
[[19357  8763  2199]
 [ 3212 54403     2]
 [    0     0 14629]]

Classification Report:
              precision    recall  f1-score   support

        city       0.86      0.64      0.73     30319
   mountains       0.86      0.94      0.90     57617
       water       0.87      1.00      0.93     14629

    accuracy                           0.86    102565
   macro avg       0.86      0.86      0.85    102565
weighted avg       0.86      0.86      0.86    102565

